In [504]:
setwd('../codice_produzione/codici/')

In [554]:
source('SME_run_model_asis.R')

Using 'N' as value column. Use 'value.var' to override




 AUC: 0.7239474 
[1] "no overfitting"
[1] "calcolo feature importance"
[1] "calcolo performance"


In [103]:
table(master_modello$DSI_presenza_fidi_ced)


                  0 0.00482826731043564                   1                   2 
              36367               52178                  20                  44 
                  3                   4                   5 
                 17                   3                   1 

In [196]:
dt_sample_limsoc <- merge(dt_results,
                         dt_orig[,.(NUM_PRATICA, PROGRESS_PRATICA,LIMITE_MAX,ESITO_FINALE_VPC)],
                         by =c('NUM_PRATICA', 'PROGRESS_PRATICA'),
                         all.x = TRUE)

dt_sample_limsoc <- merge(dt_sample_limsoc,
                        master_modello[,.(NUM_PRATICA, PROGRESS_PRATICA, Classificazione_Cliente,DSI_importo_fido_richiesto)],
                        by = c('NUM_PRATICA', 'PROGRESS_PRATICA'),
                        all.x = TRUE)

dt_sample_limsoc[,new_limiti := LIMITE_MAX]
dt_sample_limsoc[grepl("I",RAT_classe_rating_filt),new_limiti := 500000]

dt_sample_limsoc[DSI_importo_fido_richiesto >new_limiti ,new_col_limiti := "GIALLO"]
dt_sample_limsoc[DSI_importo_fido_richiesto <= new_limiti ,new_col_limiti := "VERDE"]
dt_sample_limsoc[LIMITE_MAX == 0 ,new_col_limiti := "ROSSO"]
dt_sample_limsoc[DSI_importo_fido_richiesto == 0 ,new_col_limiti := "VERDE"]
dt_sample_limsoc[grepl("R",RAT_classe_rating_filt) & RAT_classe_rating_filt!= 'UR',new_col_limiti := "ROSSO"]


dt_sample_limsoc[, new_num_limiti := as.numeric(factor(new_col_limiti, levels = c('NERO','GRIGIO','VERDE','GIALLO','ROSSO')))]

colnames(dt_sample_limsoc)

dt_sample_limsoc[, num_colnew_finale := do.call(pmax, 
                                    c(.SD, list(na.rm = TRUE))), .SDcols=c('tree_col_num','new_num_limiti')]
dt_sample_limsoc[num_colnew_finale == 3,col_finale_newvpc := "VERDE"]
dt_sample_limsoc[num_colnew_finale == 4,col_finale_newvpc := "GIALLO"]
dt_sample_limsoc[num_colnew_finale >4,col_finale_newvpc := "ROSSO"]

[1] "NUM_PRATICA"                     "PROGRESS_PRATICA"               
 [3] "AT_eta_soc"                      "AT_cod_microsett"               
 [5] "CR_sconf_acc"                    "CR_sconf_acc_mlt"               
 [7] "CR_uti_acc"                      "CR_uti_acc_revoca"              
 [9] "BIL_PFN_Ebitda_adj"              "CRA_andamento"                  
[11] "CRA_ADJ"                         "DSI_flag_scaduto_20_outstanding"
[13] "DSI_presenza_fidi_ced"           "DSI_rapporti_in_osservazione"   
[15] "AT_cod_sett_cedente"             "AT_cod_sett"                    
[17] "DSI_tipo_pratica"                "EWS_ASIS"                       
[19] "RAT_classe_rating_filt"          "FORMA_GIURIDICA"                
[21] "RAT_andamento"                   "target"                         
[23] "macrosegmento_attuale_lc"        "pred_vpc"                       
[25] "target2"                         "col_new_vpc"                    
[27] "tree_col_num"                    "LIMITE_MAX"                     
[29] "ESITO_FINALE_VPC"                "Classificazione_Cliente"        
[31] "DSI_importo_fido_richiesto"      "new_limiti"                     
[33] "new_col_limiti"                  "new_num_limiti"

In [197]:
cols_color <- c(colnames(dt_orig)[grepl("Colore -",colnames(dt_orig)) & !grepl('LIMITI SOCIETARI',colnames(dt_orig))])

In [198]:
dt_color <- dt_orig[,c("NUM_PRATICA","PROGRESS_PRATICA","ESITO_FINALE_VPC",cols_color), with = FALSE]

dt_color[, (cols_color) := lapply(dt_color[,cols_color, with = FALSE], function(col){
        as.numeric(factor(col, levels = c('NERO','GRIGIO','VERDE','GIALLO','ROSSO')))
})]

dt_color[, col_senzalimiti := do.call(pmax, c(.SD, list(na.rm = TRUE))), .SDcols=cols_color]

dt_sample_limsoc <- merge(dt_sample_limsoc,
                         dt_color[, c("NUM_PRATICA","PROGRESS_PRATICA",'col_senzalimiti')],
                         by = c("NUM_PRATICA","PROGRESS_PRATICA"),
                         all.x = TRUE)
dt_sample_limsoc[col_senzalimiti ==3, colore_senza_limsoc := "VERDE"]
dt_sample_limsoc[col_senzalimiti ==4, colore_senza_limsoc := "GIALLO"]
dt_sample_limsoc[col_senzalimiti ==5, colore_senza_limsoc := "ROSSO"]
dt_sample_limsoc[is.na(col_senzalimiti), colore_senza_limsoc := ESITO_FINALE_VPC]
dt_sample_limsoc[is.na(col_senzalimiti), col_senzalimiti := 0]

dt_sample_limsoc[, esito_finale_new_lim_num := do.call(pmax, c(.SD, list(na.rm = TRUE))), .SDcols=c('col_senzalimiti','new_num_limiti')]

dt_sample_limsoc[esito_finale_new_lim_num ==3, esito_finale_vpc_new_lim := "VERDE"]
dt_sample_limsoc[esito_finale_new_lim_num ==4, esito_finale_vpc_new_lim := "GIALLO"]
dt_sample_limsoc[esito_finale_new_lim_num ==5, esito_finale_vpc_new_lim := "ROSSO"]
dt_sample_limsoc[esito_finale_new_lim_num ==0, esito_finale_vpc_new_lim := ESITO_FINALE_VPC]


In [201]:
dt_sample_limsoc[esito_finale_new_lim_num ==0,]

Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


NUM_PRATICA,PROGRESS_PRATICA,AT_eta_soc,AT_cod_microsett,CR_sconf_acc,CR_sconf_acc_mlt,CR_uti_acc,CR_uti_acc_revoca,BIL_PFN_Ebitda_adj,CRA_andamento,⋯,DSI_importo_fido_richiesto,new_limiti,new_col_limiti,new_num_limiti,num_colnew_finale,col_finale_newvpc,col_senzalimiti,colore_senza_limsoc,esito_finale_new_lim_num,esito_finale_vpc_new_lim
<dbl>,<int>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,⋯,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>


In [202]:
table(dt_sample_limsoc[esito_finale_new_lim_num== 0,ESITO_FINALE_VPC])

#dt_sample_limsoc[esito_finale_new_lim_num ==0, esito_finale_new_lim_num := ifelse(ESITO_FINALE_VPC =='GIALLO', 4,3)]

< table of extent 0 >

In [203]:
dt_sample_limsoc[CRA_ADJ != 'missing', col_CRA_ADJ := ifelse( CRA_ADJ != "VERDE", 4,3 ) ]

dt_sample_limsoc[, num_colnew_finale_cra := do.call(pmax, 
                                    c(.SD, list(na.rm = TRUE))), .SDcols=c('col_CRA_ADJ','num_colnew_finale')]
dt_sample_limsoc[num_colnew_finale_cra == 3,col_finale_cra := "VERDE"]
dt_sample_limsoc[num_colnew_finale_cra == 4,col_finale_cra := "GIALLO"]
dt_sample_limsoc[num_colnew_finale_cra >4,col_finale_cra := "ROSSO"]

In [204]:
dt_sample_limsoc <- merge(dt_sample_limsoc,
                         dt_orig[,.(NUM_PRATICA, PROGRESS_PRATICA, `Colore - AUTONOMIA FIDO`)],
                         by = c('NUM_PRATICA', 'PROGRESS_PRATICA'),
                        all.x = TRUE)


dt_sample_limsoc[!is.na(`Colore - AUTONOMIA FIDO`), col_autonomia_fido := ifelse(`Colore - AUTONOMIA FIDO` == 'GIALLO',4,3)]


dt_sample_limsoc[, num_colnew_finale_cra_autonomia := do.call(pmax, 
                                    c(.SD, list(na.rm = TRUE))), .SDcols=c('col_autonomia_fido','num_colnew_finale_cra')]

dt_sample_limsoc[num_colnew_finale_cra_autonomia == 3,col_finale_cra_auto := "VERDE"]
dt_sample_limsoc[num_colnew_finale_cra_autonomia == 4,col_finale_cra_auto := "GIALLO"]
dt_sample_limsoc[num_colnew_finale_cra_autonomia >4,col_finale_cra_auto := "ROSSO"]

In [365]:
dt_sample_limsoc[esito_finale_new_lim_num == 3,col_old_vpc_newlim_nocra := "VERDE"]
dt_sample_limsoc[esito_finale_new_lim_num == 4,col_old_vpc_newlim_nocra := "GIALLO"]
dt_sample_limsoc[esito_finale_new_lim_num >4,col_old_vpc_newlim_nocra := "ROSSO"]

In [205]:
dt_sample_limsoc[, num_old_vpc_newlim_cra := do.call(pmax, 
                                    c(.SD, list(na.rm = TRUE))), .SDcols=c('esito_finale_new_lim_num','col_CRA_ADJ')]

dt_sample_limsoc[num_old_vpc_newlim_cra == 3,col_old_vpc_newlim_cra := "VERDE"]
dt_sample_limsoc[num_old_vpc_newlim_cra == 4,col_old_vpc_newlim_cra := "GIALLO"]
dt_sample_limsoc[num_old_vpc_newlim_cra >4,col_old_vpc_newlim_cra := "ROSSO"]

In [206]:
table(dt_sample_limsoc$esito_finale_new_lim_num )


    3     4     5 
12498  2268  1791 

In [366]:
print("performance old vpc senza cra")
rbindlist(lapply(c("VERDE","GIALLO","ROSSO"),function(x){
       dt_appo <- dt_sample_limsoc[col_old_vpc_newlim_nocra == x,
                 list(colore = x,
                      DEFAULT = nrow(dt_sample_limsoc[target=='DEFAULT' &col_old_vpc_newlim_nocra==x ,]),
                      OK = nrow(dt_sample_limsoc[target=='OK' &col_old_vpc_newlim_nocra==x ,]),
                      tot = nrow(dt_sample_limsoc[col_old_vpc_newlim_nocra==x ,]))]
       dt_appo[,td := round(100*DEFAULT/tot,3)]
       dt_appo[,sample_perc := round(100*tot/nrow(dt_sample_limsoc),3)]
       })
   )

[1] "performance old vpc senza cra"


colore,DEFAULT,OK,tot,td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
VERDE,389,12109,12498,3.112,75.485
GIALLO,68,2200,2268,2.998,13.698
ROSSO,124,1667,1791,6.924,10.817


In [207]:
print("performance old vpc")
rbindlist(lapply(c("VERDE","GIALLO","ROSSO"),function(x){
       dt_appo <- dt_sample_limsoc[col_old_vpc_newlim_cra == x,
                 list(colore = x,
                      DEFAULT = nrow(dt_sample_limsoc[target=='DEFAULT' &col_old_vpc_newlim_cra==x ,]),
                      OK = nrow(dt_sample_limsoc[target=='OK' &col_old_vpc_newlim_cra==x ,]),
                      tot = nrow(dt_sample_limsoc[col_old_vpc_newlim_cra==x ,]))]
       dt_appo[,td := round(100*DEFAULT/tot,3)]
       dt_appo[,sample_perc := round(100*tot/nrow(dt_sample_limsoc),3)]
       })
   )

[1] "performance old vpc"


colore,DEFAULT,OK,tot,td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
VERDE,362,11300,11662,3.104,70.435
GIALLO,95,3009,3104,3.061,18.747
ROSSO,124,1667,1791,6.924,10.817


In [208]:
table_cra <- rbindlist(lapply(c("VERDE","GIALLO","ROSSO"),function(x){
       dt_appo <- dt_sample_limsoc[col_finale_cra == x,
                 list(colore = x,
                      DEFAULT = nrow(dt_sample_limsoc[target=='DEFAULT' &col_finale_cra==x ,]),
                      OK = nrow(dt_sample_limsoc[target=='OK' &col_finale_cra==x ,]),
                      tot = nrow(dt_sample_limsoc[col_finale_cra==x ,]))]
       dt_appo[,td := round(100*DEFAULT/tot,3)]
       dt_appo[,sample_perc := round(100*tot/nrow(dt_sample_limsoc),3)]
       })
   )
table_cra

colore,DEFAULT,OK,tot,td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
VERDE,234,11692,11926,1.962,72.030
GIALLO,138,2719,2857,4.830,17.256
ROSSO,209,1565,1774,11.781,10.715


In [209]:
table_cra_auto <- rbindlist(lapply(c("VERDE","GIALLO","ROSSO"),function(x){
              dt_appo <- dt_sample_limsoc[col_finale_cra_auto == x,
                        list(colore = x,
                             DEFAULT = nrow(dt_sample_limsoc[target=='DEFAULT' &col_finale_cra_auto==x ,]),
                             OK = nrow(dt_sample_limsoc[target=='OK' &col_finale_cra_auto==x ,]),
                             tot = nrow(dt_sample_limsoc[col_finale_cra_auto==x ,]))]
              dt_appo[,td := round(100*DEFAULT/tot,3)]
              dt_appo[,sample_perc := round(100*tot/nrow(dt_sample_limsoc),3)]
              })
          )
table_cra_auto

colore,DEFAULT,OK,tot,td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
VERDE,230,11524,11754,1.957,70.991
GIALLO,142,2887,3029,4.688,18.294
ROSSO,209,1565,1774,11.781,10.715


In [210]:
nrow(test_clienti) + nrow(test_nonclienti)
nrow(dt_sample_limsoc)

[1] 16557

[1] 16557

In [368]:
test_nonclienti <- dt_sample_limsoc[Classificazione_Cliente == 'NON CLIENTE']
test_clienti <- dt_sample_limsoc[Classificazione_Cliente == 'CLIENTE']
#test_clienti

In [369]:
rbindlist(lapply(c("VERDE","GIALLO","ROSSO"),function(x){
       dt_appo <- dt_sample_limsoc[col_finale_newvpc == x,
                 list(colore = x,
                      DEFAULT = nrow(dt_sample_limsoc[target=='DEFAULT' &col_finale_newvpc==x ,]),
                      OK = nrow(dt_sample_limsoc[target=='OK' &col_finale_newvpc==x ,]),
                      tot = nrow(dt_sample_limsoc[col_finale_newvpc==x ,]))]
       dt_appo[,td := round(100*DEFAULT/tot,3)]
       dt_appo[,sample_perc := round(100*tot/nrow(dt_sample_limsoc),3)]
       })
   )

colore,DEFAULT,OK,tot,td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
VERDE,265,12709,12974,2.043,78.360
GIALLO,107,1702,1809,5.915,10.926
ROSSO,209,1565,1774,11.781,10.715


In [370]:
print('perfomance non clienti pre cra ')
table_nonclienti <- rbindlist(lapply(c("VERDE","GIALLO","ROSSO"),function(x){
       dt_appo <- test_nonclienti[col_finale_newvpc == x,
                 list(colore = x,
                      DEFAULT = nrow(test_nonclienti[target=='DEFAULT' &col_finale_newvpc==x ,]),
                      OK = nrow(test_nonclienti[target=='OK' &col_finale_newvpc==x ,]),
                      tot = nrow(test_nonclienti[col_finale_newvpc==x ,]))]
       dt_appo[,td := round(100*DEFAULT/tot,3)]
       dt_appo[,sample_perc := round(100*tot/nrow(test_nonclienti),3)]
       })
   )
table_nonclienti

[1] "perfomance non clienti pre cra "


colore,DEFAULT,OK,tot,td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
VERDE,215,8418,8633,2.490,79.883
GIALLO,91,1311,1402,6.491,12.973
ROSSO,83,689,772,10.751,7.144


In [371]:
print('perfomance clienti pre cra ')
table_clienti <- rbindlist(lapply(c("VERDE","GIALLO","ROSSO"),function(x){
       dt_appo <- test_clienti[col_finale_newvpc == x,
                 list(colore = x,
                      DEFAULT = nrow(test_clienti[target=='DEFAULT' &col_finale_newvpc==x ,]),
                      OK = nrow(test_clienti[target=='OK' &col_finale_newvpc==x ,]),
                      tot = nrow(test_clienti[col_finale_newvpc==x ,]))]
       dt_appo[,td := round(100*DEFAULT/tot,3)]
       dt_appo[,sample_perc := round(100*tot/nrow(test_clienti),3)]
       })
   )
table_clienti

[1] "perfomance clienti pre cra "


colore,DEFAULT,OK,tot,td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
VERDE,50,4291,4341,1.152,75.496
GIALLO,16,391,407,3.931,7.078
ROSSO,126,876,1002,12.575,17.426


In [215]:
table_cra_nonclienti <- rbindlist(lapply(c("VERDE","GIALLO","ROSSO"),function(x){
       dt_appo <- test_nonclienti[col_finale_cra == x,
                 list(colore = x,
                      DEFAULT = nrow(test_nonclienti[target=='DEFAULT' &col_finale_cra==x ,]),
                      OK = nrow(test_nonclienti[target=='OK' &col_finale_cra==x ,]),
                      tot = nrow(test_nonclienti[col_finale_cra==x ,]))]
       dt_appo[,td := round(100*DEFAULT/tot,3)]
       dt_appo[,sample_perc := round(100*tot/nrow(test_nonclienti),3)]
       })
   )
table_cra_nonclienti

colore,DEFAULT,OK,tot,td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
VERDE,201,7981,8182,2.457,75.710
GIALLO,105,1748,1853,5.666,17.146
ROSSO,83,689,772,10.751,7.144


In [216]:
table_cra_clienti <- rbindlist(lapply(c("VERDE","GIALLO","ROSSO"),function(x){
       dt_appo <- test_clienti[col_finale_cra == x,
                 list(colore = x,
                      DEFAULT = nrow(test_clienti[target=='DEFAULT' &col_finale_cra==x ,]),
                      OK = nrow(test_clienti[target=='OK' &col_finale_cra==x ,]),
                      tot = nrow(test_clienti[col_finale_cra==x ,]))]
       dt_appo[,td := round(100*DEFAULT/tot,3)]
       dt_appo[,sample_perc := round(100*tot/nrow(test_clienti),3)]
       })
   )
table_cra_clienti

colore,DEFAULT,OK,tot,td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
VERDE,33,3711,3744,0.881,65.113
GIALLO,33,971,1004,3.287,17.461
ROSSO,126,876,1002,12.575,17.426


In [376]:
print('vpc asis (vecchi limiti) clienti')
rbindlist(lapply(c("VERDE","GIALLO","ROSSO"),function(x){
       dt_appo <- test_clienti[ESITO_FINALE_VPC == x,
                 list(colore = x,
                      DEFAULT = nrow(test_clienti[target=='DEFAULT' &ESITO_FINALE_VPC==x ,]),
                      OK = nrow(test_clienti[target=='OK' &ESITO_FINALE_VPC==x ,]),
                      old_tot = nrow(test_clienti[ESITO_FINALE_VPC==x ,]))]
       dt_appo[,old_td := round(100*DEFAULT/old_tot,3)]
       dt_appo[,sample_perc := round(100*old_tot/nrow(test_clienti),3)]
       })
   )

[1] "vpc asis (vecchi limiti) clienti"


colore,DEFAULT,OK,old_tot,old_td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
VERDE,79,3689,3768,2.097,65.530
GIALLO,46,1114,1160,3.966,20.174
ROSSO,67,755,822,8.151,14.296


In [375]:
print('vpc asis (vecchi limiti) non clienti')
rbindlist(lapply(c("VERDE","GIALLO","ROSSO"),function(x){
       dt_appo <- test_nonclienti[ESITO_FINALE_VPC == x,
                 list(colore = x,
                      DEFAULT = nrow(test_nonclienti[target=='DEFAULT' &ESITO_FINALE_VPC==x ,]),
                      OK = nrow(test_nonclienti[target=='OK' &ESITO_FINALE_VPC==x ,]),
                      old_tot = nrow(test_nonclienti[ESITO_FINALE_VPC==x ,]))]
       dt_appo[,old_td := round(100*DEFAULT/old_tot,3)]
       dt_appo[,sample_perc := round(100*old_tot/nrow(test_nonclienti),3)]
       })
   )

[1] "vpc asis (vecchi limiti) non clienti"


colore,DEFAULT,OK,old_tot,old_td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
VERDE,337,8945,9282,3.631,85.889
GIALLO,29,838,867,3.345,8.023
ROSSO,23,635,658,3.495,6.089


In [218]:
colnames(test_nonclienti)
table_nonclienti_oldvpc <- rbindlist(lapply(c("VERDE","GIALLO","ROSSO"),function(x){
       dt_appo <- test_nonclienti[col_old_vpc_newlim_cra == x,
                 list(colore = x,
                      DEFAULT = nrow(test_nonclienti[target=='DEFAULT' &col_old_vpc_newlim_cra==x ,]),
                      OK = nrow(test_nonclienti[target=='OK' &col_old_vpc_newlim_cra==x ,]),
                      old_tot = nrow(test_nonclienti[col_old_vpc_newlim_cra==x ,]))]
       dt_appo[,old_td := round(100*DEFAULT/old_tot,3)]
       dt_appo[,sample_perc := round(100*old_tot/nrow(test_nonclienti),3)]
       })
   )
table_nonclienti_oldvpc

[1] "NUM_PRATICA"                     "PROGRESS_PRATICA"               
 [3] "AT_eta_soc"                      "AT_cod_microsett"               
 [5] "CR_sconf_acc"                    "CR_sconf_acc_mlt"               
 [7] "CR_uti_acc"                      "CR_uti_acc_revoca"              
 [9] "BIL_PFN_Ebitda_adj"              "CRA_andamento"                  
[11] "CRA_ADJ"                         "DSI_flag_scaduto_20_outstanding"
[13] "DSI_presenza_fidi_ced"           "DSI_rapporti_in_osservazione"   
[15] "AT_cod_sett_cedente"             "AT_cod_sett"                    
[17] "DSI_tipo_pratica"                "EWS_ASIS"                       
[19] "RAT_classe_rating_filt"          "FORMA_GIURIDICA"                
[21] "RAT_andamento"                   "target"                         
[23] "macrosegmento_attuale_lc"        "pred_vpc"                       
[25] "target2"                         "col_new_vpc"                    
[27] "tree_col_num"                    "LIMITE_MAX"                     
[29] "ESITO_FINALE_VPC"                "Classificazione_Cliente"        
[31] "DSI_importo_fido_richiesto"      "new_limiti"                     
[33] "new_col_limiti"                  "new_num_limiti"                 
[35] "num_colnew_finale"               "col_finale_newvpc"              
[37] "col_senzalimiti"                 "colore_senza_limsoc"            
[39] "esito_finale_new_lim_num"        "esito_finale_vpc_new_lim"       
[41] "col_CRA_ADJ"                     "num_colnew_finale_cra"          
[43] "col_finale_cra"                  "Colore - AUTONOMIA FIDO"        
[45] "col_autonomia_fido"              "num_colnew_finale_cra_autonomia"
[47] "col_finale_cra_auto"             "num_old_vpc_newlim_cra"         
[49] "col_old_vpc_newlim_cra"

colore,DEFAULT,OK,old_tot,old_td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
VERDE,319,8466,8785,3.631,81.290
GIALLO,39,1244,1283,3.040,11.872
ROSSO,31,708,739,4.195,6.838


In [373]:
print("performance old vpc senza cra CLIENTI")
rbindlist(lapply(c("VERDE","GIALLO","ROSSO"),function(x){
       dt_appo <- test_clienti[col_old_vpc_newlim_nocra == x,
                 list(colore = x,
                      DEFAULT = nrow(test_clienti[target=='DEFAULT' &col_old_vpc_newlim_nocra==x ,]),
                      OK = nrow(test_clienti[target=='OK' &col_old_vpc_newlim_nocra==x ,]),
                      tot = nrow(test_clienti[col_old_vpc_newlim_nocra==x ,]))]
       dt_appo[,td := round(100*DEFAULT/tot,3)]
       dt_appo[,sample_perc := round(100*tot/nrow(test_clienti),3)]
       })
   )

[1] "performance old vpc senza cra CLIENTI"


colore,DEFAULT,OK,tot,td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
VERDE,57,3265,3322,1.716,57.774
GIALLO,42,1334,1376,3.052,23.930
ROSSO,93,959,1052,8.840,18.296


In [374]:
print("performance old vpc senza cra NON CLIENTI")
rbindlist(lapply(c("VERDE","GIALLO","ROSSO"),function(x){
       dt_appo <- test_nonclienti[col_old_vpc_newlim_nocra == x,
                 list(colore = x,
                      DEFAULT = nrow(test_nonclienti[target=='DEFAULT' &col_old_vpc_newlim_nocra==x ,]),
                      OK = nrow(test_nonclienti[target=='OK' &col_old_vpc_newlim_nocra==x ,]),
                      tot = nrow(test_nonclienti[col_old_vpc_newlim_nocra==x ,]))]
       dt_appo[,td := round(100*DEFAULT/tot,3)]
       dt_appo[,sample_perc := round(100*tot/nrow(test_nonclienti),3)]
       })
   )

[1] "performance old vpc senza cra NON CLIENTI"


colore,DEFAULT,OK,tot,td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
VERDE,332,8844,9176,3.618,84.908
GIALLO,26,866,892,2.915,8.254
ROSSO,31,708,739,4.195,6.838


In [219]:

table_clienti_oldvpc <- rbindlist(lapply(c("VERDE","GIALLO","ROSSO"),function(x){
       dt_appo <- test_clienti[col_old_vpc_newlim_cra == x,
                 list(colore = x,
                      DEFAULT = nrow(test_clienti[target=='DEFAULT' &col_old_vpc_newlim_cra==x ,]),
                      OK = nrow(test_clienti[target=='OK' &col_old_vpc_newlim_cra==x ,]),
                      old_tot = nrow(test_clienti[col_old_vpc_newlim_cra==x ,]))]
       dt_appo[,old_td := round(100*DEFAULT/old_tot,3)]
       dt_appo[,sample_perc := round(100*old_tot/nrow(test_clienti),3)]
       })
   )
table_clienti_oldvpc

colore,DEFAULT,OK,old_tot,old_td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
VERDE,43,2834,2877,1.495,50.035
GIALLO,56,1765,1821,3.075,31.670
ROSSO,93,959,1052,8.840,18.296


In [220]:
write.xlsx(table_nonclienti, 'risultati_nonclienti_sme.xlsx',sheetName = 'precra', append = FALSE)
write.xlsx(table_cra_nonclienti, 'risultati_nonclienti_sme.xlsx',sheetName = 'postcra', append = TRUE)
write.xlsx(table_nonclienti_oldvpc, 'risultati_nonclienti_sme.xlsx',sheetName = 'vpc_asis', append = TRUE)

In [145]:
write.xlsx(table_cra_auto, 'risultati_postlimauto.xlsx',sheetName = 'postcra_auto_sme', append = FALSE)
write.xlsx(table_cra, 'risultati_postlimauto.xlsx',sheetName = 'postcra_sme', append = TRUE)

### ESTRAZIONE RAMI PRINCIPALI

In [86]:
model_frame_foglie <- model_frame[ type != "other"]
model_frame_foglie[, perc := round(100*n/sum(n),1)]


dt_esempi <- rbindlist(lapply(unique(dt_sample$pred_vpc), function(s){
    dt <- head(dt_sample[pred_vpc == s,],1)
}))

dt_rules <- setDT(unique(rpart.predict(modello_smer$model,newdata= dt_esempi, rules = TRUE)))

colnames(dt_rules) <- c('DEFAULT',"OK","ramo")
dt_rules <- dt_rules[,.(DEFAULT,ramo)]

dt_rules[, nodi := str_count(ramo, "&")+1]
dt_rules[,ramo := gsub("\\s+"," ",ramo)]
#dt_rules[,ramo := gsub("&","& ____",ramo)]
#dt_rules[,ramo := gsub("____","\n\n",ramo)]

In [87]:
dt_rules <- merge(dt_rules,
                 model_frame_foglie[,.(yval2.V4,perc)],
                 by.x = 'DEFAULT',
                 by.y = 'yval2.V4',
                 all.x = TRUE)


dt_rules[, microsettore := gsub('\\s+'," ",gsub('[a-z,&]',"",str_extract_all(ramo, "microsett is .+&")))]

mapping_settori <- unique(master_modello[,.(AT_sett_desc,AT_cod_microsett)])

dt_rules <- rbindlist(lapply((1:nrow(dt_rules)),function(i){
    dt<- dt_rules[i,]
     dt[,settore_desc := paste(unique(mapping_settori[AT_cod_microsett %in% unlist(strsplit(microsettore," ")),AT_sett_desc]),collapse = "; ")]
#     dt
    
}))

write.xlsx(dt_rules[order(-perc)],'sme/tabella_rami_sme.xlsx')

write.csv(dt_rules[order(-perc)],'sme/tabella_rami_sme_crarat.csv')
dt_rules

DEFAULT,ramo,nodi,perc,microsettore,settore_desc
<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>
0.01043930,because EWS_ASIS < 1.5 & AT_cod_microsett is ABCON01 or ABCON02 or ABCON03 or AEROS01 or AGRIC04 or AGRIC05 or ALIMD01 or ALIMP03 or ALIMP05 or APMED01 or APMED02 or ASERV01 or ASERV02 or ASERV03 or ASERV05 or ASERV07 or ASERV08 or ASERV11 or ASSIC01 or AUTOP01 or AUTOP02 or CHIMI01 or CHIMI02 or CHIMI04 or CHIMI06 or CHIMI07 or CHIMI08 or CHIMI09 or COMPO01 or COMPO02 or COMPO05 or COMPO06 or COSTR01 or DETTA01 or DETTA02 or DETTA04 or DIFES01 or DIFES02 or DUREP02 or DUREP03 or DUREP04 or ESTRA02 or FARMD01 or FARMD02 or FARMP01 or FARMP02 or FINSE01 or GIOAZ01 or HARDW01 or HARDW02 or HARDW03 or HARDW04 or HARDW05 or HOLDI01 or HOLDI02 or IMMOB01 or IMMOB03 or IMMOB04 or INFRA01 or INFRA02 or INFRA06 or INFRA07 or INGRO02 or ISTRU01 or ISTRU02 or LEGNO01 or MACCH01 or MAIMG01 or MAIMG02 or MAIMG03 or MAIMG04 or MAIMG05 or MAIMG06 or MAIMG07 or MAIMS01 or MAIMS02 or MAIMS03 or MAIMS04 or MAIMS05 or MAIMS07 or MAIMS08 or MATCO02 or MATCO03 or MATCO04 or MATCO06 or METAL01 or METAL03 or OILGA01 or OILGA03 or OILGA04 or OILGA06 or OILGA08 or POWER01 or PRODM01 or RACTR01 or RACTR05 or SERVA01 or SILVI01 or SOFTW01 or SOFTW02 or TABAC01 or TABAC02 or TABAC03 or TABAC04 or TESSP04 or TESSP05 or TESSP06 or TESSP07 or TRASP01 or TRASP02 or TRASS02 or TRASS03 or TRASS04 or TRASS07 or TRASS08 or TRASS09 or TRASS13 or TURAL02 or TURRI01 or missing & AT_eta_soc >= 10.6,3,32.0,ABCON01 ABCON02 ABCON03 AEROS01 AGRIC04 AGRIC05 ALIMD01 ALIMP03 ALIMP05 APMED01 APMED02 ASERV01 ASERV02 ASERV03 ASERV05 ASERV07 ASERV08 ASERV11 ASSIC01 AUTOP01 AUTOP02 CHIMI01 CHIMI02 CHIMI04 CHIMI06 CHIMI07 CHIMI08 CHIMI09 COMPO01 COMPO02 COMPO05 COMPO06 COSTR01 DETTA01 DETTA02 DETTA04 DIFES01 DIFES02 DUREP02 DUREP03 DUREP04 ESTRA02 FARMD01 FARMD02 FARMP01 FARMP02 FINSE01 GIOAZ01 HARDW01 HARDW02 HARDW03 HARDW04 HARDW05 HOLDI01 HOLDI02 IMMOB01 IMMOB03 IMMOB04 INFRA01 INFRA02 INFRA06 INFRA07 INGRO02 ISTRU01 ISTRU02 LEGNO01 MACCH01 MAIMG01 MAIMG02 MAIMG03 MAIMG04 MAIMG05 MAIMG06 MAIMG07 MAIMS01 MAIMS02 MAIMS03 MAIMS04 MAIMS05 MAIMS07 MAIMS08 MATCO02 MATCO03 MATCO04 MATCO06 METAL01 METAL03 OILGA01 OILGA03 OILGA04 OILGA06 OILGA08 POWER01 PRODM01 RACTR01 RACTR05 SERVA01 SILVI01 SOFTW01 SOFTW02 TABAC01 TABAC02 TABAC03 TABAC04 TESSP04 TESSP05 TESSP06 TESSP07 TRASP01 TRASP02 TRASS02 TRASS03 TRASS04 TRASS07 TRASS08 TRASS09 TRASS13 TURAL02 TURRI01,"Software e Servizi IT; Servizi di assistenza sanitaria; Farmaceutica e Biotecnologie - Distribuzione; Alimentare e bevande - Produzione; Altri servizi; Prodotti in metallo; Turismo - Ristorazione; Macchinari e altre attrezzature - Ingrosso; Macchine per impieghi speciali; Trasporti - Servizi; Costruzioni - Edilizia; Turismo - Alloggio e Agenzie di viaggio; Chimica, Gomma e Plastica; Legno e carta; Ingrosso - Altro; Dettaglio - Altro; Assicurazioni; Alimentare e bevande - Distribuzione; Materiali costruzioni; Oil and Gas; Infrastrutture; Tessile, abbigliamento e lusso - Produzione; Immobiliare; Silvicoltura e altre attività forestali; Financial services; Macchine per impieghi generali; Tabacco; Durevoli - Produzione; Altri beni di consumo; Gioco d'azzardo; Istruzione; Estrazione; Agricoltura, pesca e allevamento; Power Generation; Metallurgia; Hardware e Semiconduttori; Holding; Componenti e apparecchiature elettriche; Apparecchiature medicali; Raccolta e trattamento dei rifiuti; Trasporti - Produzione; Automotive - Produzione; Difesa; Farmaceutica e Biotecnologie - Produzione; Aerospace"
0.01927248,because EWS_ASIS < 1.5 & AT_cod_microsett is ABCON01 or ABCON02 or ABCON03 or AEROS01 or AGRIC04 or AGRIC05 or ALIMP05 or APMED01 or APMED02 or ASERV02 or ASERV08 or ASERV11 or ASSIC01 or AUTOP02 or CHIMI01 or CHIMI02 or CHIMI04 or CHIMI07 or CHIMI08 or DETTA02 or DETTA04 or DUREP02 or DUREP03 or DUREP04 or ESTRA02 or FARMD01 or FARMD02 or FARMP02 or FINSE01 or GIOAZ01 or HARDW01 or HARDW02 or HARDW04 or HARDW05 or HOLDI01 or HOLDI02 or IMMO

## scrittura excel performance

In [29]:
ncol(table_performance)+3

[1] 14

In [97]:
str(wb)

'data.frame':	21 obs. of  13 variables:
 $ pred_vpc  : num  0.575 0.559 0.515 0.474 0.382 ...
 $ DEFAULT   : num  9 1 0 9 4 12 30 5 8 15 ...
 $ OK        : num  11 4 3 19 14 17 103 33 50 86 ...
 $ tot       : num  20 5 3 28 18 29 133 38 58 101 ...
 $ tot_verdi : num  100 99.9 99.8 99.8 99.7 ...
 $ lift_verde: num  1 0.99 0.98 0.98 0.97 0.96 0.95 0.9 0.89 0.88 ...
 $ td_foglia : num  45 20 0 32.1 22.2 ...
 $ def_cum   : num  9 10 10 19 23 35 65 70 78 93 ...
 $ ok_cum    : num  11 15 18 37 51 68 171 204 254 340 ...
 $ tot_cum   : num  20 25 28 56 74 103 236 274 332 433 ...
 $ recall    : num  0 1.55 1.72 1.72 3.27 ...
 $ X12       : chr  "soglia_vg" "0.3" NA NA ...
 $ X13       : chr  "soglia_gr" "0.8" NA NA ...


In [122]:
wb

A Workbook object.
 
Worksheets:
 Sheet 1: "Sheet1"
 

 
 Worksheet write order: 1

In [132]:
wb$sheet_names[1]

[1] "Sheet1"

In [145]:
require(openxlsx)
start <- ncol(table_performance)+3
wb <- createWorkbook()
addWorksheet(wb, "Sheet 1")
writeData(wb, "Sheet 1", x = table_performance)
writeData(wb, sheet = "Sheet 1", x = data.table(soglia_vg= 0.3, soglia_gr = 0.8), startCol = start, startRow = 2)
writeFormula(wb, 1, x = "SUMIF(A2:A10,A2>0)", startCol = 13, startRow = 10)

saveWorkbook(wb, "performance_tool.xlsx", overwrite = TRUE)

In [141]:
getSheets(wb)

ERROR: Error in envRefInferField(x, what, getClass(class(x)), selfEnv): ‘getNumberOfSheets’ is not a valid field or method name for reference class “Workbook”


In [136]:
addDataFrame(x= data.table(prova = "prova"),getSheets(wb)$, startRow = 712)
saveWorkbook(wb,  "performance_tool.xlsx")


# #saveWorkbook(wb, "performance_tool.xlsx", overwrite = FALSE)

ERROR: Error: $ operator is invalid for atomic vectors


In [94]:
df_vero <- table_performance

In [ ]:
# rows<-getRows(sheet)[17:26]
# cells<-getCells(rows)
# cells<-cells[grepl(paste0(".",ncol(df_vero)),names(cells),fixed=TRUE)]

# for (x in seq_len(length(cells))){
    i<-gsub(paste0(".",ncol(df_vero)),"",names(cells)[x],fixed=TRUE)
    mycell<-cells[[x]]
    my_formula <- paste0(letters[ncol(df_vero)+1],"$2")
    for (y in 2:ncol(df_coeff)){
        my_add <- paste0("+",letters[ncol(df_vero)+y],'$2*',letters[y],i)
        my_formula <- paste0(my_formula,my_add)
    }
    mycell$setCellFormula(my_formula)

    evaluator$evaluateFormulaCell( mycell )   
    mycell$getNumericCellValue()


# }

In [65]:
rows<-getRows(sheet)[17:26]
                    cells<-getCells(rows)
                    cells<-cells[grepl(paste0(".",ncol(df_vero)),names(cells),fixed=TRUE)]
paste0('=+SUMIFS(B2:B',nrow(table_performance),';A2:A',nrow(table_performance),';"<="&',toupper(letters[start]),'3)')

paste0('=+SUMIFS(B2:B',nrow(table_performance),';A2:A',nrow(table_performance),';"=" 0,474226804123711')

'=+SUMIFS($B$2:$B$1048572;$A$2:$A$1048572;"<="&$R$2)'

[1] "=+SUMIFS(B2:B21;A2:A21;\"<=\"&N3)"

[1] "=+SUMIFS(B2:B21;A2:A21;\"=\" 0,474226804123711"

[1] "=+SUMIFS($B$2:$B$1048572;$A$2:$A$1048572;\"<=\"&$R$2)"

In [34]:
paste0('=+SUMIFS($B$2:$B$',nrow(table_performance),';$A$2:$A$',nrow(table_performance),';"<="&$',toupper(letters[start]),'$2)')

[1] "=+SUMIFS($B$2:$B$21;$A$2:$A$21;\"<=\"&$N$2)"

In [ ]:
v <- paste0('=+SUMIFS($B$2:$B$',nrow(table_performance),';$A$2:$A$',nrow(table_performance),';"<="&$',letters[start],'$2)') ## skip header row
writeFormula(wb, sheet = 1, x = v, startCol = 10, startRow = 2)
writeFormula(wb, 1, x = "A2 + B2", startCol = 10, startRow = 10)

In [25]:
## 2. - As a data.frame column with class "formula" using writeData

df <- data.frame(
  x = 1:3,
  y = 1:3,
  z = paste(paste0("A", 1:3 + 1L), paste0("B", 1:3 + 1L), sep = " + "),
  z2 = sprintf("ADDRESS(1,%s)", 1:3),
  stringsAsFactors = FALSE
)

class(df$z) <- c(class(df$z), "formula")
class(df$z2) <- c(class(df$z2), "formula")

addWorksheet(wb, "Sheet 2")
writeData(wb, sheet = 2, x = df)



## 3. - As a vector with class "formula" using writeData

v2 <- c("SUM(A2:A4)", "AVERAGE(B2:B4)", "MEDIAN(C2:C4)")
class(v2) <- c(class(v2), "formula")

writeData(wb, sheet = 2, x = v2, startCol = 10, startRow = 2)

saveWorkbook(wb, "performance_tool.xlsx", overwrite = TRUE)

ERROR: Error in writeFormula(wb, sheet = 1, x = v, startCol = 10, startRow = 2): non trovo la funzione "writeFormula"


In [245]:
getwd()

[1] "/Users/52495/Projects/VPC/VPC_2022/renv_codice_produzione/codici"

In [246]:
dt_sme <- setDT(read.csv('../master/sme/storico_17_21.csv'))


In [247]:
colnames(dt_sme)

[1] "X"                                       
  [2] "NUM_PRATICA"                             
  [3] "PROGRESS_PRATICA"                        
  [4] "LEVA_check"                              
  [5] "LEVA_FLAG"                               
  [6] "SOST_DEB_FLAG"                           
  [7] "INDICATORE_SOSTENIBILITA_DEBITO"         
  [8] "AFFORDABILITY_M3"                        
  [9] "AFFORDABILITY_M3_FLAG"                   
 [10] "AT_eta_soc"                              
 [11] "AT_cod_sett"                             
 [12] "AT_cod_microsett"                        
 [13] "AT_cod_sett_cedente"                     
 [14] "AT_cod_microsett_cedente"                
 [15] "AT_sett_desc"                            
 [16] "AT_microsett_desc"                       
 [17] "AT_sett_desc_cedente"                    
 [18] "AT_microsett_desc_cedente"               
 [19] "BIL_fatturato"                           
 [20] "BIL_pat_netto"                           
 [21] "BIL_PFN_Ebitda"                          
 [22] "BIL_onerifin_ebitda"                     
 [23] "BIL_delta_fatturato"                     
 [24] "BIL_delta_pat_netto"                     
 [25] "PFN"                                     
 [26] "Ebitda"                                  
 [27] "BIL_PFN_Ebitda_adj"                      
 [28] "BIL_MFISC_fatturato"                     
 [29] "BIL_MFISC_pat_netto"                     
 [30] "BIL_MFISC_delta_fatturato"               
 [31] "BIL_MFISC_delta_pat_netto"               
 [32] "Classificazione_Cliente"                 
 [33] "CR_accordato"                            
 [34] "CR_sconfino"                             
 [35] "CR_utilizzato"                           
 [36] "CR_accordato_revoca"                     
 [37] "CR_sconfino_revoca"                      
 [38] "CR_utilizzato_revoca"                    
 [39] "CR_accordato_autoliq"                    
 [40] "CR_sconfino_autoliq"                     
 [41] "CR_utilizzato_autoliq"                   
 [42] "CR_accordato_scadenza"                   
 [43] "CR_sconfino_scadenza"                    
 [44] "CR_utilizzato_scadenza"                  
 [45] "CR_accordato_bt"                         
 [46] "CR_sconfino_bt"                          
 [47] "CR_utilizzato_bt"                        
 [48] "CR_accordato_mlt"                        
 [49] "CR_sconfino_mlt"                         
 [50] "CR_utilizzato_mlt"                       
 [51] "CR_sconf_acc"                            
 [52] "CR_sconf_acc_bt"                         
 [53] "CR_sconf_acc_mlt"                        
 [54] "CR_uti_acc"                              
 [55] "CR_uti_acc_bt"                           
 [56] "CR_uti_acc_mlt"                          
 [57] "CR_uti_acc_revoca"                       
 [58] "CR_uti_acc_autoliq"                      
 [59] "CR_uti_acc_scadenza"                     
 [60] "CRA_andamento"                           
 [61] "DSI_tipo_pratica"                        
 [62] "DSI_filiale_factoring_ceduto"            
 [63] "DSI_settore_ceduto"                      
 [64] "DSI_divisione_ceduto"                    
 [65] "DSI_importo_richiesto_applicazione"      
 [66] "DSI_settore_cedente"                     
 [67] "DSI_tipo_prodotto"                       
 [68] "DSI_rae_ceduto"                          
 [69] "DSI_polizza_diretta_39_"                 
 [70] "DSI_polizza_indiretta_41_"               
 [71] "macrosegmento_attuale_lc"                
 [72] "target"                                  
 [73] "DSI_sab"                                 
 [74] "DSI_col_presenza_pratica_pef"            
 [75] "DSI_col_fido_con_causale_negativa"       
 [76] "DSI_col_appl_con_causale_negativa"       
 [77] "DSI_parte_correlata"                     
 [78] "DSI_rapporti_in_osservazione"            
 [79] "DSI_causale_negativa_fido"               
 [80] "DSI_diff_appr_appl"                      
 [81] "DSI_num_fidi_scaduti"                    
 [82] "DSI_flag_fidi_scaduti"    

In [248]:
dt_small <- dt_sme[flag_exante == 'NO',..cols_modello]

In [249]:
write.csv(dt_small,'../master/sme/storico_17_21.csv',row.names = FALSE )



dt_sme2 <- setDT(read.csv('../master/sme/storico_17_21.csv'))

dt_sme2#[,..cols_modello]

AT_eta_soc,AT_cod_microsett,CR_sconf_acc,CR_sconf_acc_mlt,CR_uti_acc,CR_uti_acc_revoca,BIL_PFN_Ebitda_adj,CRA_andamento,CRA_ADJ,DSI_flag_scaduto_20_outstanding,⋯,AT_cod_sett,DSI_tipo_pratica,EWS_ASIS,NUM_PRATICA,PROGRESS_PRATICA,RAT_classe_rating_filt,FORMA_GIURIDICA,RAT_andamento,target,macrosegmento_attuale_lc
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,⋯,<chr>,<int>,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
4.72,SOFTW02,0,-10000,2.498602e-02,1.396369e-01,0.000000e+00,Stabile,VERDE,non censito,⋯,SOFTW00,3,0.000000,102230026,1,I,SRL,Stabile,OK,Sme Retail
7.81,SERVA01,-10000,-10000,-1.000000e+04,-1.000000e+04,-3.967542e+04,missing,missing,non censito,⋯,SERVA00,3,0.283975,102230029,2,missing,SCRL,missing,OK,Sme Retail
17.98,COSTR02,-10000,-10000,-1.000000e+04,-1.000000e+04,-4.275362e-01,Stabile,VERDE,FALSE,⋯,COSTR00,2,0.000000,102230034,1,M,SRL,Stabile,OK,Sme Retail
19.03,ASERV04,-10000,-10000,-1.000000e+04,-1.000000e+04,-3.905325e-01,missing,VERDE,FALSE,⋯,ASERV00,3,0.283975,102230035,1,I,SRL,missing,OK,Sme Retail
17.65,FARMD01,-10000,-10000,-1.000000e+04,-1.000000e+04,-1.735294e+00,missing,missing,FALSE,⋯,FARMD00,3,0.283975,103230027,1,missing,SRL,missing,OK,Sme Retail
41.02,AGRIC01,-10000,-10000,-1.000000e+04,-1.000000e+04,3.413043e+00,Cra non presente ultimo trimestre,VERDE,FALSE,⋯,AGRIC00,3,0.000000,103230031,1,M,SCRL,Non Utilizzabile UR,OK,Sme Retail
14.54,ALIMD02,0,0,6.904651e-01,5.337433e-01,5.687898e+00,Stabile,VERDE,FALSE,⋯,ALIMD00,3,0.000000,103230033,1,I,SRL,Peggiorato,OK,Sme Retail
31.10,ALIMP03,0,0,8.015916e-01,6.556532e-01,-3.230769e+00,Stabile,VERDE,FALSE,⋯,ALIMP00,3,0.000000,103230034,1,M,SRL,Peggiorato,OK,Sme Retail
8.01,TRASS10,-10000,-10000,-1.000000e+04,-1.000000e+04,4.734940e+00,Cra non presente ultimo trimestre,VERDE,FALSE,⋯,TRASS00,3,0.000000,103230039,1,M,SRL,Non Utilizzabile UR,OK,Sme Retail


In [308]:
master_nfill <- readRDS('../../data/RDS/master/master_sme_0707.RDS')

In [310]:
master_nfill <- master_nfill[flag_exante == "NO",]
pred_mod<-predict(modello,master_nfill[-rows,])
pred_mod<- as.data.table(pred_mod)

In [311]:
test$pred_vpc_2 <- as.numeric(pred_mod$DEFAULT)

In [312]:
length(unique(test$pred_vpc_2))
length(unique(test$pred_vpc))

[1] 21

[1] 21

In [313]:
master_nfill[-rows,..cols_modello]

AT_eta_soc,AT_cod_microsett,CR_sconf_acc,CR_sconf_acc_mlt,CR_uti_acc,CR_uti_acc_revoca,BIL_PFN_Ebitda_adj,CRA_andamento,CRA_ADJ,DSI_flag_scaduto_20_outstanding,⋯,AT_cod_sett,DSI_tipo_pratica,EWS_ASIS,NUM_PRATICA,PROGRESS_PRATICA,RAT_classe_rating_filt,FORMA_GIURIDICA,RAT_andamento,target,macrosegmento_attuale_lc
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
10.62,TRASS10,-1.000000e+04,-10000,-1.000000e+04,-1.000000e+04,-29.6923077,Cra non presente ultimo trimestre,VERDE,FALSE,⋯,TRASS00,3,0,103230046,1,M,SRL,Non Utilizzabile UR,OK,Sme Retail
16.02,TRASS10,-1.000000e+04,-10000,-1.000000e+04,-1.000000e+04,NA,Cra non presente ultimo trimestre,VERDE,FALSE,⋯,TRASS00,3,0,107230033,1,I,DI,Non Utilizzabile UR,OK,Sme Retail
37.74,PRODM01,-1.000000e+04,-10000,-1.000000e+04,-1.000000e+04,-1.7248577,NA,NA,FALSE,⋯,PRODM00,3,0,108230022,1,NA,SRL,NA,OK,Sme Retail
11.49,PRODM01,-1.000000e+04,-10000,-1.000000e+04,-1.000000e+04,2.0168067,Stabile,NA,FALSE,⋯,PRODM00,3,0,108230032,1,NA,SRL,Stabile,OK,Sme Retail
10.05,AUTOD01,0.000000e+00,0,6.898441e-01,0.000000e+00,3.0102564,Peggiorato,NA,FALSE,⋯,AUTOD00,3,0,108230033,1,NA,SRL,Stabile,OK,Sme Retail
7.86,MAIMS08,8.945193e-04,0,6.674511e-01,2.330981e-01,11111.0000000,Stabile,VERDE,FALSE,⋯,MAIMS00,3,0,109230036,1,M,SRL,Stabile,OK,Sme Retail
35.20,TRASS10,-1.000000e+04,-10000,-1.000000e+04,-1.000000e+04,NA,Stabile,VERDE,FALSE,⋯,TRASS00,3,0,109230046,1,I,DI,Stabile,OK,Sme Retail
19.47,CHIMI06,0.000000e+00,0,6.041083e-01,9.656471e-02,5.7272727,Stabile,VERDE,FALSE,⋯,CHIMI00,3,0,113230036,4,M,SRL,Stabile,OK,Sme Retail
3.96,ASERV09,-1.000000e+04,-10000,-1.000000e+04,-1.000000e+04,0.0000000,NA,GIALLO,FALSE,⋯,ASERV00,3,NA,113230044,1,M,SRL,NA,OK,Sme Retail


In [314]:
test1 <- master_nfill[-rows,..cols_modello][is.na(RAT_classe_rating_filt) ,]


In [315]:
pred_mod<-predict(modello,test1)
pred_mod<- as.data.table(pred_mod)

test1$pred_vpc <- as.numeric(pred_mod$DEFAULT)

In [316]:
test1

AT_eta_soc,AT_cod_microsett,CR_sconf_acc,CR_sconf_acc_mlt,CR_uti_acc,CR_uti_acc_revoca,BIL_PFN_Ebitda_adj,CRA_andamento,CRA_ADJ,DSI_flag_scaduto_20_outstanding,⋯,DSI_tipo_pratica,EWS_ASIS,NUM_PRATICA,PROGRESS_PRATICA,RAT_classe_rating_filt,FORMA_GIURIDICA,RAT_andamento,target,macrosegmento_attuale_lc,pred_vpc
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,⋯,<chr>,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
37.74,PRODM01,-10000,-10000,-1.000000e+04,-10000,-1.7248577,NA,NA,FALSE,⋯,3,0,108230022,1,NA,SRL,NA,OK,Sme Retail,0.01043930
11.49,PRODM01,-10000,-10000,-1.000000e+04,-10000,2.0168067,Stabile,NA,FALSE,⋯,3,0,108230032,1,NA,SRL,Stabile,OK,Sme Retail,0.01043930
10.05,AUTOD01,0,0,6.898441e-01,0,3.0102564,Peggiorato,NA,FALSE,⋯,3,0,108230033,1,NA,SRL,Stabile,OK,Sme Retail,0.03750000
39.48,ALIMD02,-10000,-10000,-1.000000e+04,-10000,NA,NA,NA,non censito,⋯,3,NA,115230029,1,NA,DI,NA,OK,Sme Retail,0.02607731
16.84,ALIMP03,-10000,-10000,-1.000000e+04,-10000,NA,NA,NA,FALSE,⋯,3,NA,115230030,1,NA,DI,NA,OK,Sme Retail,0.01043930
0.91,DURED01,-10000,-10000,-1.000000e+04,-10000,NA,NA,NA,non censito,⋯,3,NA,116231732,1,NA,SRL,NA,OK,Sme Retail,0.09047120
12.25,TRASS10,-10000,-10000,-1.000000e+04,-10000,NA,Cra non presente ultimo trimestre,NA,non censito,⋯,3,NA,116231733,2,NA,DI,Non Utilizzabile UR,OK,Sme Retail,0.02607731
0.50,DURED01,-10000,-10000,-1.000000e+04,-10000,NA,NA,NA,non censito,⋯,3,NA,117230035,1,NA,SRL,NA,OK,Sme Retail,0.09047120
13.15,PRODM01,-10000,-10000,-1.000000e+04,-10000,-0.2293578,Cra non presente ultimo trimestre,NA,non censito,⋯,3,NA,117230060,4,NA,SRL,Non Utilizzabile UR,DEFAULT,Sme Retail,0.01043930


In [317]:
table(test1$pred_vpc, test1$target)

                    
                     DEFAULT   OK
  0.0104393008974965      56 3329
  0.0192724793951788      71 2819
  0.0260773071943777      39 1584
  0.0315789473684211       7   73
  0.0375                  20  250
  0.0425014042314173      55 1083
  0.0622710622710623      14   86
  0.080338266384778        8   20
  0.0904712041884817      81 1038
  0.116279069767442        0    1
  0.117918313570488       32  220
  0.129716981132075        3    4
  0.150406504065041        6   30
  0.219251336898396        5   22
  0.256124721603563       26   73
  0.258426966292135        2    4
  0.381818181818182        1    9
  0.474226804123711        6   17
  0.558823529411765        1    2
  0.575342465753425        1    3

In [318]:
table(test1[pred_vpc > 0.095  ,target == 'OK'])


FALSE  TRUE 
   83   385 

In [320]:
master_nfill[-rows,..cols_modello]

AT_eta_soc,AT_cod_microsett,CR_sconf_acc,CR_sconf_acc_mlt,CR_uti_acc,CR_uti_acc_revoca,BIL_PFN_Ebitda_adj,CRA_andamento,CRA_ADJ,DSI_flag_scaduto_20_outstanding,⋯,AT_cod_sett,DSI_tipo_pratica,EWS_ASIS,NUM_PRATICA,PROGRESS_PRATICA,RAT_classe_rating_filt,FORMA_GIURIDICA,RAT_andamento,target,macrosegmento_attuale_lc
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
10.62,TRASS10,-1.000000e+04,-10000,-1.000000e+04,-1.000000e+04,-29.6923077,Cra non presente ultimo trimestre,VERDE,FALSE,⋯,TRASS00,3,0,103230046,1,M,SRL,Non Utilizzabile UR,OK,Sme Retail
16.02,TRASS10,-1.000000e+04,-10000,-1.000000e+04,-1.000000e+04,NA,Cra non presente ultimo trimestre,VERDE,FALSE,⋯,TRASS00,3,0,107230033,1,I,DI,Non Utilizzabile UR,OK,Sme Retail
37.74,PRODM01,-1.000000e+04,-10000,-1.000000e+04,-1.000000e+04,-1.7248577,NA,NA,FALSE,⋯,PRODM00,3,0,108230022,1,NA,SRL,NA,OK,Sme Retail
11.49,PRODM01,-1.000000e+04,-10000,-1.000000e+04,-1.000000e+04,2.0168067,Stabile,NA,FALSE,⋯,PRODM00,3,0,108230032,1,NA,SRL,Stabile,OK,Sme Retail
10.05,AUTOD01,0.000000e+00,0,6.898441e-01,0.000000e+00,3.0102564,Peggiorato,NA,FALSE,⋯,AUTOD00,3,0,108230033,1,NA,SRL,Stabile,OK,Sme Retail
7.86,MAIMS08,8.945193e-04,0,6.674511e-01,2.330981e-01,11111.0000000,Stabile,VERDE,FALSE,⋯,MAIMS00,3,0,109230036,1,M,SRL,Stabile,OK,Sme Retail
35.20,TRASS10,-1.000000e+04,-10000,-1.000000e+04,-1.000000e+04,NA,Stabile,VERDE,FALSE,⋯,TRASS00,3,0,109230046,1,I,DI,Stabile,OK,Sme Retail
19.47,CHIMI06,0.000000e+00,0,6.041083e-01,9.656471e-02,5.7272727,Stabile,VERDE,FALSE,⋯,CHIMI00,3,0,113230036,4,M,SRL,Stabile,OK,Sme Retail
3.96,ASERV09,-1.000000e+04,-10000,-1.000000e+04,-1.000000e+04,0.0000000,NA,GIALLO,FALSE,⋯,ASERV00,3,NA,113230044,1,M,SRL,NA,OK,Sme Retail


In [321]:
test2 <- master_nfill[-rows,..cols_modello][!is.na(RAT_classe_rating_filt) ,]

In [329]:
test2[CR_sconf_acc == -10000, CR_sconf_acc := NA]
test2[CR_sconf_acc_mlt == -10000, CR_sconf_acc_mlt := NA]
test2[CR_uti_acc == -10000, CR_uti_acc := NA]
test2[CR_uti_acc_revoca == -10000, CR_uti_acc_revoca := NA]

In [330]:
pred_mod<-predict(modello,test2)
pred_mod<- as.data.table(pred_mod)

test2$pred_vpc <- as.numeric(pred_mod$DEFAULT)

In [332]:
test2[, colore1 :="GIALLO"]

test2[pred_vpc >= soglia_gr, colore1 :="ROSSO"]
test2[pred_vpc < soglia_vg, colore1 :="VERDE"]

In [333]:
table(test2$colore1)


GIALLO  ROSSO  VERDE 
   117    183   5156 

In [352]:
test3 <- copy(test2)

In [353]:
test3$RAT_classe_rating_filt <- NA
test3$CRA_ADJ <- NA
test3$RAT_andamento <- NA
test3$CRA_andamento <- NA
test3[is.na(EWS_ASIS),EWS_ASIS :=  mean(master_nfill$EWS_ASIS, na.rm = TRUE)]

In [354]:
pred_mod<-predict(modello,test3)
pred_mod<- as.data.table(pred_mod)

test3$pred_vpc <- as.numeric(pred_mod$DEFAULT)

In [355]:
test3[, colore2 :="GIALLO"]

test3[pred_vpc >= soglia_gr, colore2 :="ROSSO"]
test3[pred_vpc < soglia_vg, colore2 :="VERDE"]

In [356]:
#test3

In [357]:
table(test3$EWS_ASIS, useNA = 'always')


               0 0.23924131335671                1                2 
            2511             2535              151              152 
               3                4                5             <NA> 
             104                2                1                0 

In [362]:
test3[colore1 =='VERDE' & colore2 == 'ROSSO', ]

AT_eta_soc,AT_cod_microsett,CR_sconf_acc,CR_sconf_acc_mlt,CR_uti_acc,CR_uti_acc_revoca,BIL_PFN_Ebitda_adj,CRA_andamento,CRA_ADJ,DSI_flag_scaduto_20_outstanding,⋯,NUM_PRATICA,PROGRESS_PRATICA,RAT_classe_rating_filt,FORMA_GIURIDICA,RAT_andamento,target,macrosegmento_attuale_lc,pred_vpc,colore1,colore2
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,⋯,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
3.96,ASERV09,NA,NA,NA,NA,0.00000000,NA,NA,FALSE,⋯,113230044,1,NA,SRL,NA,OK,Sme Retail,0.2561247,VERDE,ROSSO
4.08,GDO0001,NA,NA,NA,NA,NA,NA,NA,FALSE,⋯,217230093,1,NA,DI,NA,OK,Sme Retail,0.1179183,VERDE,ROSSO
6.17,ALIMD02,NA,NA,NA,NA,7.27272727,NA,NA,FALSE,⋯,218230162,1,NA,SRL,NA,OK,Sme Retail,0.2561247,VERDE,ROSSO
2.24,ALIMD01,NA,NA,NA,NA,NA,NA,NA,FALSE,⋯,310230377,1,NA,DI,NA,OK,Sme Retail,0.2192513,VERDE,ROSSO
6.36,TRASS10,0,0,0.57421454,0.4444444,-1.50000000,NA,NA,FALSE,⋯,522230048,1,NA,SRL,NA,OK,Sme Retail,0.1179183,VERDE,ROSSO
10.05,TRASS10,NA,NA,NA,NA,0.82558140,NA,NA,FALSE,⋯,527230042,1,NA,DI,NA,OK,Sme Retail,0.1179183,VERDE,ROSSO
10.50,COSTR01,NA,NA,NA,NA,NA,NA,NA,FALSE,⋯,812230041,1,NA,DI,NA,OK,Sme Retail,0.2192513,VERDE,ROSSO
4.23,TRASS10,NA,NA,NA,NA,NA,NA,NA,FALSE,⋯,813230042,1,NA,DI,NA,OK,Sme Retail,0.1179183,VERDE,ROSSO
9.49,COSTR04,0,0,0.93363849,0.7094956,NA,NA,NA,FALSE,⋯,818230018,1,NA,DI,NA,OK,Sme Retail,0.4742268,VERDE,ROSSO


In [344]:
cols_modello

[1] "AT_eta_soc"                      "AT_cod_microsett"               
 [3] "CR_sconf_acc"                    "CR_sconf_acc_mlt"               
 [5] "CR_uti_acc"                      "CR_uti_acc_revoca"              
 [7] "BIL_PFN_Ebitda_adj"              "CRA_andamento"                  
 [9] "CRA_ADJ"                         "DSI_flag_scaduto_20_outstanding"
[11] "DSI_presenza_fidi_ced"           "DSI_rapporti_in_osservazione"   
[13] "AT_cod_sett_cedente"             "AT_cod_sett"                    
[15] "DSI_tipo_pratica"                "EWS_ASIS"                       
[17] "NUM_PRATICA"                     "PROGRESS_PRATICA"               
[19] "RAT_classe_rating_filt"          "FORMA_GIURIDICA"                
[21] "RAT_andamento"                   "target"                         
[23] "macrosegmento_attuale_lc"

## tabella costruzione variabili

In [506]:
tabella_variabili <- rbindlist(lapply(colnames(master_modello[,..cols_modello]), function(col){
    if(class(master_modello[,get(col)]) == 'factor'){
        valori <- paste(unique(master_modello[,get(col)]),collapse = ", ")
    }
    else{
        valori <- ""
    }
    data.table(variabile = col, tipo = class(master_modello[,get(col)]),esempi = valori)
}))

write.xlsx(tabella_variabili,'tabella_variabili_sme.xlsx', row.names = FALSE)

In [507]:
getwd()

[1] "/Users/52495/Projects/VPC/VPC_2022/renv_codice_produzione/codice_produzione/codici"

## read pmml

In [541]:
dt_esempi <- rbindlist(lapply(unique(test$pred_vpc), function(s){
    dt <- head(test[pred_vpc == s,],1)
}))

dt_rules <- setDT(unique(rpart.predict(modello,newdata= dt_esempi, rules = TRUE)))
colnames(dt_rules) <- c('DEFAULT',"OK","ramo")
dt_rules <- dt_rules[,.(DEFAULT,ramo)]

dt_rules[, nodi := str_count(ramo, "&")+1]

dt_rules[ , round_pred := round(DEFAULT,2)]

In [561]:
dt_rules

DEFAULT,ramo,nodi,round_pred
<dbl>,<chr>,<dbl>,<dbl>
0.02607731,because EWS_ASIS < 1.5 & AT_cod_microsett is ABCON04 or AGRIC01 or AGRIC02 or ALIMD02 or ALIMP04 or ASERV04 or ASERV06 or ASERV09 or ASERV10 or ATRIC01 or ATRIC02 or ATRIC03 or AUTOD01 or CHIMI03 or COMPO03 or COMPO04 or COSTR02 or COSTR04 or DETTA03 or DURED01 or GDO0001 or IMMOB02 or INFRA05 or INGRO01 or MAIMS06 or MATCO01 or MATCO05 or METAL02 or OILGA05 or RACTR02 or RACTR03 or RACTR04 or RACTR06 or TELEC02 or TESSD01 or TESSD02 or TESSP03 or TRASS06 or TRASS10 or TRASS11 or TRASS12 or TURAL01 or UTILI01 or UTILI02 & AT_eta_soc >= 10.5,3,0.03
0.01043930,because EWS_ASIS < 1.5 & AT_cod_microsett is ABCON01 or ABCON02 or ABCON03 or AEROS01 or AGRIC04 or AGRIC05 or ALIMD01 or ALIMP03 or ALIMP05 or APMED01 or APMED02 or ASERV01 or ASERV02 or ASERV03 or ASERV05 or ASERV07 or ASERV08 or ASERV11 or ASSIC01 or AUTOP01 or AUTOP02 or CHIMI01 or CHIMI02 or CHIMI04 or CHIMI06 or CHIMI07 or CHIMI08 or CHIMI09 or COMPO01 or COMPO02 or COMPO05 or COMPO06 or COSTR01 or DETTA01 or DETTA02 or DETTA04 or DIFES01 or DIFES02 or DUREP02 or DUREP03 or DUREP04 or ESTRA02 or FARMD01 or FARMD02 or FARMP01 or FARMP02 or FINSE01 or GIOAZ01 or HARDW01 or HARDW02 or HARDW03 or HARDW04 or HARDW05 or HOLDI01 or HOLDI02 or IMMOB01 or IMMOB03 or IMMOB04 or INFRA01 or INFRA02 or INFRA06 or INFRA07 or INGRO02 or ISTRU01 or ISTRU02 or LEGNO01 or MACCH01 or MAIMG01 or MAIMG02 or MAIMG03 or MAIMG04 or MAIMG05 or MAIMG06 or MAIMG07 or MAIMS01 or MAIMS02 or MAIMS03 or MAIMS04 or MAIMS05 or MAIMS07 or MAIMS08 or MATCO02 or MATCO03 or MATCO04 or MATCO06 or METAL01 or METAL03 or OILGA01 or OILGA03 or OILGA04 or OILGA06 or OILGA08 or POWER01 or PRODM01 or RACTR01 or RACTR05 or SERVA01 or SILVI01 or SOFTW01 or SOFTW02 or TABAC01 or TABAC02 or TABAC03 or TABAC04 or TESSP04 or TESSP05 or TESSP06 or TESSP07 or TRASP01 or TRASP02 or TRASS02 or TRASS03 or TRASS04 or TRASS07 or TRASS08 or TRASS09 or TRASS13 or TURAL02 or TURRI01 or missing & AT_eta_soc >= 10.6,3,0.01
0.11791831,because EWS_ASIS < 1.5 & AT_cod_microsett is AGRIC02 or ASERV04 or ATRIC02 or ATRIC03 or AUTOD01 or CHIMI03 or COMPO03 or COMPO04 or DURED01 or GDO0001 or INFRA05 or INGRO01 or MAIMS06 or MATCO01 or METAL02 or RACTR03 or TRASS06 or TRASS10 or TRASS11 or TURAL01 or UTILI01 & AT_eta_soc < 10.5 & CRA_ADJ is ROSSO or missing & DSI_presenza_fidi_ced < 0.0024,5,0.12
0.01927248,because EWS_ASIS < 1.5 & AT_cod_microsett is ABCON01 or ABCON02 or ABCON03 or AEROS01 or AGRIC04 or AGRIC05 or ALIMP05 or APMED01 or APMED02 or ASERV02 or ASERV08 or ASERV11 or ASSIC01 or AUTOP02 or CHIMI01 or CHIMI02 or CHIMI04 or CHIMI07 or CHIMI08 or DETTA02 or DETTA04 or DUREP02 or DUREP03 or DUREP04 or ESTRA02 or FARMD01 or FARMD02 or FARMP02 or FINSE01 or GIOAZ01 or HARDW01 or HARDW02 or HARDW04 or HARDW05 or HOLDI01 or HOLDI02 or IMMOB04 or INFRA06 or ISTRU01 or ISTRU02 or MAIMG01 or MAIMG03 or MAIMG04 or MAIMG06 or MAIMG07 or MAIMS03 or MAIMS04 or MAIMS05 or MAIMS07 or MAIMS08 or MATCO02 or MATCO04 or MATCO06 or METAL01 or METAL03 or OILGA01 or OILGA03 or OILGA08 or POWER01 or RACTR01 or RACTR05 or TABAC02 or TABAC04 or TESSP05 or TESSP06 or TESSP07 or TRASP01 or TRASP02 or TRASS03 or TRASS08 or TURRI01 & AT_eta_soc < 10.6,3,0.02
0.03750000,because EWS_ASIS < 1.5 & AT_cod_microsett is ABCON04 or AGRIC01 or AGRIC02 or ALIMD02 or ALIMP04 or ASERV04 or ASERV06 or ASERV09 or ASERV10 or ATRIC01 or ATRIC02 or ATRIC03 or AUTOD01 or CHIMI03 or COMPO03 or COMPO04 or COSTR02 or COSTR04 or DETTA03 or DURED01 or GDO0001 or IMMOB02 or INFRA05 or INGRO01 or MAIMS06 or MATCO01 or MATCO05 or METAL02 or OILGA05 or RACTR02 or RACTR03 or RACTR04 or RACTR06 or TELEC02 or TESSD01 or TESSD02 or TESSP03 or TRASS06 or TRASS10 or TRASS11 or TRASS12 or TURAL01 or UTILI01 or UTILI02 & AT_eta_soc < 10.5 & CRA_ADJ is GIALLO or VERDE,4,0.04
0.09047120,because EWS_ASIS < 1.5 & AT_cod_microsett is ABCON04 or AGRIC01 or AGRIC02 or ALIMD02 or ALIMP04 or ASERV04 or ASERV06 or ASERV09 or ASERV10 o

In [563]:
require(readr)

pmml <- read_file("~/Desktop/risultati_0630/sme/sme_tree.xml")
pmml <- gsub('.*</Output>(.+)</TreeModel>.*', "\\1",pmml)
dt_nodi <- data.table(nodo = unlist(strsplit(pmml, ' <Node id=')))
dt_nodi[, surrogate:= grepl('surrogate',nodo)]

dt_nodi$surrogate_vars <- unlist(lapply((1:nrow(dt_nodi)), function(i){
   surr_vars <- paste(unlist(stringi::stri_extract_all_regex(dt_nodi[i,nodo], '(?<=field=").*?(?=")')),collapse = ';  ')
}))



dt_nodi$pred <- unlist(lapply((1:nrow(dt_nodi)), function(i){
   surr_vars <- round(as.numeric(unlist(stringi::stri_extract_all_regex(dt_nodi[i,nodo], '(?<=confidence=").*?(?=")'))[1]),2)
}))

dt_nodi$nod_num <- unlist(lapply((1:nrow(dt_nodi)), function(i){
   surr_vars <- paste(unlist(stringi::stri_extract_all_regex(dt_nodi[i,nodo], '(?<=").*?(?= score=")'))[1],collapse = ';  ')
}))

dt_nodi

nodo                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [564]:
dt_nodi <- merge(dt_nodi,
                dt_rules[,.(round_pred, nodi)],
                by.x = 'pred',
                by.y = 'round_pred',
                all.x = TRUE)

In [584]:
dt_nodi[order(nodi)]#[startsWith(surrogate_vars, "RAT_"),]

pred
1  0.08
2  0.13
3  0.13
4  0.52
5  0.01
6  0.02
7  0.02
8  0.03
9  0.03
10 0.03
11 0.03
12 0.03
13 0.12
14 0.12
15 0.58
16 0.04
17 0.04
18 0.04
19 0.04
20 0.26
21 0.26
22 0.56
23 0.03
24 0.03
25 0.03
26 0.03
27 0.03
28 0.09
29 0.09
30 0.09
31 0.09
32 0.12
33 0.12
34 0.38
35 0.22
36 0.26
37 0.26
38 0.47
39 0.06
40 0.06
41 0.15
42   NA
43 0.05
44 0.11
45 0.11
46 0.16
47 0.16
48 0.17
49 0.23
50 0.29
51 0.34
52 0.37
53 0.43
   nodo                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [489]:
write.xlsx(dt_nodi,'tabella_nodi_sme.xlsx', row.names = FALSE)

In [490]:
getwd()

[1] "/Users/52495/Projects/VPC/VPC_2022/renv_codice_produzione/codici"

In [494]:
colnames(master_modello)

[1] "X"                                       
  [2] "NUM_PRATICA"                             
  [3] "PROGRESS_PRATICA"                        
  [4] "LEVA_check"                              
  [5] "LEVA_FLAG"                               
  [6] "SOST_DEB_FLAG"                           
  [7] "INDICATORE_SOSTENIBILITA_DEBITO"         
  [8] "AFFORDABILITY_M3"                        
  [9] "AFFORDABILITY_M3_FLAG"                   
 [10] "AT_eta_soc"                              
 [11] "AT_cod_sett"                             
 [12] "AT_cod_microsett"                        
 [13] "AT_cod_sett_cedente"                     
 [14] "AT_cod_microsett_cedente"                
 [15] "AT_sett_desc"                            
 [16] "AT_microsett_desc"                       
 [17] "AT_sett_desc_cedente"                    
 [18] "AT_microsett_desc_cedente"               
 [19] "BIL_fatturato"                           
 [20] "BIL_pat_netto"                           
 [21] "BIL_PFN_Ebitda"                          
 [22] "BIL_onerifin_ebitda"                     
 [23] "BIL_delta_fatturato"                     
 [24] "BIL_delta_pat_netto"                     
 [25] "PFN"                                     
 [26] "Ebitda"                                  
 [27] "BIL_PFN_Ebitda_adj"                      
 [28] "BIL_MFISC_fatturato"                     
 [29] "BIL_MFISC_pat_netto"                     
 [30] "BIL_MFISC_delta_fatturato"               
 [31] "BIL_MFISC_delta_pat_netto"               
 [32] "Classificazione_Cliente"                 
 [33] "CR_accordato"                            
 [34] "CR_sconfino"                             
 [35] "CR_utilizzato"                           
 [36] "CR_accordato_revoca"                     
 [37] "CR_sconfino_revoca"                      
 [38] "CR_utilizzato_revoca"                    
 [39] "CR_accordato_autoliq"                    
 [40] "CR_sconfino_autoliq"                     
 [41] "CR_utilizzato_autoliq"                   
 [42] "CR_accordato_scadenza"                   
 [43] "CR_sconfino_scadenza"                    
 [44] "CR_utilizzato_scadenza"                  
 [45] "CR_accordato_bt"                         
 [46] "CR_sconfino_bt"                          
 [47] "CR_utilizzato_bt"                        
 [48] "CR_accordato_mlt"                        
 [49] "CR_sconfino_mlt"                         
 [50] "CR_utilizzato_mlt"                       
 [51] "CR_sconf_acc"                            
 [52] "CR_sconf_acc_bt"                         
 [53] "CR_sconf_acc_mlt"                        
 [54] "CR_uti_acc"                              
 [55] "CR_uti_acc_bt"                           
 [56] "CR_uti_acc_mlt"                          
 [57] "CR_uti_acc_revoca"                       
 [58] "CR_uti_acc_autoliq"                      
 [59] "CR_uti_acc_scadenza"                     
 [60] "CRA_andamento"                           
 [61] "DSI_tipo_pratica"                        
 [62] "DSI_filiale_factoring_ceduto"            
 [63] "DSI_settore_ceduto"                      
 [64] "DSI_divisione_ceduto"                    
 [65] "DSI_importo_richiesto_applicazione"      
 [66] "DSI_settore_cedente"                     
 [67] "DSI_tipo_prodotto"                       
 [68] "DSI_rae_ceduto"                          
 [69] "DSI_polizza_diretta_39_"                 
 [70] "DSI_polizza_indiretta_41_"               
 [71] "macrosegmento_attuale_lc"                
 [72] "target"                                  
 [73] "DSI_sab"                                 
 [74] "DSI_col_presenza_pratica_pef"            
 [75] "DSI_col_fido_con_causale_negativa"       
 [76] "DSI_col_appl_con_causale_negativa"       
 [77] "DSI_parte_correlata"                     
 [78] "DSI_rapporti_in_osservazione"            
 [79] "DSI_causale_negativa_fido"               
 [80] "DSI_diff_appr_appl"                      
 [81] "DSI_num_fidi_scaduti"                    
 [82] "DSI_flag_fidi_scaduti"    

In [ ]:
master_modello[!is.na(master_modello$BIL_PFN_Ebitda_adj,)]

In [496]:
getwd()

[1] "/Users/52495/Projects/VPC/VPC_2022/renv_codice_produzione/codici"

In [500]:
test_2save <- copy(test)
test_2save[,DSI_tipo_pratica := as.character(DSI_tipo_pratica) ]

write.csv(test_2save, '../codice_produzione/data/test/sme/dt_prova.csv')

In [502]:

dt_results_check <- copy(test)
dt_results_check[pred_vpc > soglia_vg & pred_vpc <=soglia_gr, col_new_vpc := "giallo"]
dt_results_check[pred_vpc <= soglia_vg, col_new_vpc := "verde"]
dt_results_check[pred_vpc > soglia_gr, col_new_vpc := "rosso"]

table_results_check <- rbindlist(lapply(c("verde","giallo","rosso"),function(x){
   dt_appo <- dt_results_check[col_new_vpc == x,
             list(colore = x,
                  DEFAULT = nrow(dt_results_check[target=='DEFAULT' &col_new_vpc==x ,]),
                  OK = nrow(dt_results_check[target=='OK' &col_new_vpc==x ,]),
                  tot = nrow(dt_results_check[col_new_vpc==x ,]))]
   dt_appo[,td := round(100*DEFAULT/tot,3)]
   dt_appo[,sample_perc := round(100*tot/nrow(dt_results_check),3)]
   })
)

In [503]:
table_results_check

colore,DEFAULT,OK,tot,td,sample_perc
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
verde,329,14096,14425,2.281,87.123
giallo,115,1225,1340,8.582,8.093
rosso,137,655,792,17.298,4.783


## TEST

In [509]:
table(test$DSI_presenza_fidi_ced)


0.0104393008974965 0.0192724793951788 0.0260773071943777 0.0315789473684211 
              5220               4090               2661                195 
            0.0375 0.0425014042314173 0.0622710622710623  0.080338266384778 
               735               1393                131                132 
0.0904712041884817  0.116279069767442  0.117918313570488  0.129716981132075 
              1208                 17                342                101 
 0.150406504065041  0.219251336898396  0.256124721603563  0.258426966292135 
                58                 38                133                 29 
 0.381818181818182  0.474226804123711  0.515151515151515  0.558823529411765 
                18                 28                  3                  5 
 0.575342465753425 
                20 

In [518]:
table(test[pred_vpc <=0.05 & pred_vpc >=0.039,])


   2    3 
   1 1392 

## ANALISI BILANCIO